Библиотеки

In [3]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

Скачивание и сохранения датасета

In [ ]:
# Загружаем CIFAR-100 из Keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)

# Создадим базовую папку для датасета
base_dir = "cifar100_raw"
os.makedirs(base_dir, exist_ok=True)

# Сохраняем в виде изображений по классам
def save_dataset(images, labels, split):
    split_dir = os.path.join(base_dir, split)
    os.makedirs(split_dir, exist_ok=True)
    for idx, (img, label) in enumerate(zip(images, labels)):
        class_id = int(label[0])
        class_dir = os.path.join(split_dir, str(class_id))
        os.makedirs(class_dir, exist_ok=True)
        im = Image.fromarray(img)
        im.save(os.path.join(class_dir, f"{idx}.png"))

save_dataset(x_train, y_train, "train")
save_dataset(x_test, y_test, "test")

print("Датасет сохранён в папку cifar100_raw/")


Аугментация данных

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Генератор с аугментациями
datagen = ImageDataGenerator(
    rotation_range=15,       # случайные повороты
    width_shift_range=0.1,   # сдвиги по ширине
    height_shift_range=0.1,  # сдвиги по высоте
    horizontal_flip=True,    # отражения
    zoom_range=0.1           # масштабирование
)

# Папка для сохранения аугментированных данных
aug_dir = "cifar100_augmented"
os.makedirs(aug_dir, exist_ok=True)

def augment_and_save(split="train", samples_per_image=2):
    src_dir = os.path.join(base_dir, split)
    for class_id in os.listdir(src_dir):
        class_src = os.path.join(src_dir, class_id)
        class_dst = os.path.join(aug_dir, split, class_id)
        os.makedirs(class_dst, exist_ok=True)

        for img_name in os.listdir(class_src):
            img_path = os.path.join(class_src, img_name)
            img = Image.open(img_path)
            x = np.expand_dims(np.array(img), axis=0)  # (1,h,w,3)

            # создаём N аугментированных версий
            i = 0
            for batch in datagen.flow(x, batch_size=1):
                im = Image.fromarray(batch[0].astype("uint8"))
                im.save(os.path.join(class_dst, f"{img_name[:-4]}_aug{i}.png"))
                i += 1
                if i >= samples_per_image:  # сколько копий на одно изображение
                    break

augment_and_save("train", samples_per_image=2)
print("Аугментация сохранена в cifar100_augmented/")


Обучение модели EfficientNetV2B0

In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU list:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.20.0
Num GPUs Available: 0
GPU list: []


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0

img_size = (224,224)  # EfficientNet требует увеличенных изображений
batch_size = 16
num_classes = 100

# Загружаем данные из сохранённой структуры директорий
train_ds = tf.keras.utils.image_dataset_from_directory(
    "cifar100_augmented/train",
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "cifar100_raw/test",
    image_size=img_size,
    batch_size=batch_size
)

# Кеширование и prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Data augmentation (runtime, для доп.разнообразия)
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Базовая модель EfficientNetV2B0
base_model = EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3)
)
base_model.trainable = False  # замораживаем backbone

# Строим модель
inputs = layers.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

# Компиляция
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Обучение (первый этап, только классификатор)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# Fine-tuning (разморозим backbone)
base_model.trainable = True
for layer in base_model.layers[:-50]:  # разморозим только верхние 50 слоёв
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Found 100000 files belonging to 100 classes.
Found 10000 files belonging to 100 classes.
Epoch 1/10
2363/3125 ━━━━━━━━━━━━━━━━━━━━ 1:37 128ms/step - accuracy: 0.3783 - loss: 2.5722

: 